In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import json
import random
import pandas as pd
import shutil

<IPython.core.display.Javascript object>

In [3]:
!mkdir prepare_data/img_clf_multilabel_lst/train_imgs
!mkdir prepare_data/img_clf_multilabel_lst/valid_imgs
!mkdir prepare_data/img_clf_multilabel_lst/train_annots
!mkdir prepare_data/img_clf_multilabel_lst/valid_annots

mkdir: prepare_data/img_clf_multilabel_lst/train_imgs: File exists
mkdir: prepare_data/img_clf_multilabel_lst/valid_imgs: File exists
mkdir: prepare_data/img_clf_multilabel_lst/train_annots: File exists
mkdir: prepare_data/img_clf_multilabel_lst/valid_annots: File exists


<IPython.core.display.Javascript object>

In [4]:
gt_df = pd.read_csv(
    "prepare_data/img_clf_multilabel_lst/all_images_gt/clf_labels.csv", header=None
)
gt_df.columns = ["image", "class"]
gt_df.head()

,image,class
0,9314850.jpg,"[banana,apple]"
1,6799149.jpg,[apple]
2,9741355.jpg,"[banana,apple]"
3,3811265.jpg,"[orange,banana,apple]"
4,5260672.jpg,"[orange,banana]"


<IPython.core.display.Javascript object>

In [5]:
train_valid_split = 0.7
nimages = gt_df["image"].nunique()
ntrain = int(train_valid_split * nimages)
nvalid = nimages - ntrain
print(nimages, ntrain, nvalid)

100 70 30


<IPython.core.display.Javascript object>

In [6]:
gt_df["class"] = gt_df["class"].apply(lambda x: x[1:-1])
gt_df.head()

,image,class
0,9314850.jpg,"banana,apple"
1,6799149.jpg,apple
2,9741355.jpg,"banana,apple"
3,3811265.jpg,"orange,banana,apple"
4,5260672.jpg,"orange,banana"


<IPython.core.display.Javascript object>

In [7]:
labels = ["apple", "orange", "banana"]
labels_map = {k: v for v, k in enumerate(labels)}
with open("prepare_data/img_clf_multilabel_lst/labels_map.json", "w") as fp:
    json.dump(labels_map, fp)
labels_map

{'apple': 0, 'orange': 1, 'banana': 2}

<IPython.core.display.Javascript object>

In [8]:
def function(x):
    xl = x.split(",")
    xll = []
    for lbl in labels:
        if lbl in xl:
            xll.append(1)
        else:
            xll.append(0)
    return xll

<IPython.core.display.Javascript object>

In [9]:
gt_df["class2"] = gt_df["class"].apply(function)
gt_df.head()

,image,class,class2
0,9314850.jpg,"banana,apple","[1, 0, 1]"
1,6799149.jpg,apple,"[1, 0, 0]"
2,9741355.jpg,"banana,apple","[1, 0, 1]"
3,3811265.jpg,"orange,banana,apple","[1, 1, 1]"
4,5260672.jpg,"orange,banana","[0, 1, 1]"


<IPython.core.display.Javascript object>

In [10]:
gt_df[labels] = gt_df["class2"].apply(pd.Series)
gt_df.head()

,image,class,class2,apple,orange,banana
0,9314850.jpg,"banana,apple","[1, 0, 1]",1,0,1
1,6799149.jpg,apple,"[1, 0, 0]",1,0,0
2,9741355.jpg,"banana,apple","[1, 0, 1]",1,0,1
3,3811265.jpg,"orange,banana,apple","[1, 1, 1]",1,1,1
4,5260672.jpg,"orange,banana","[0, 1, 1]",0,1,1


<IPython.core.display.Javascript object>

In [11]:
gt_df = gt_df.sample(frac=1).reset_index(drop=True)
gt_df.head()

,image,class,class2,apple,orange,banana
0,5553944.jpg,banana,"[0, 0, 1]",0,0,1
1,9985835.jpg,"apple,orange,banana","[1, 1, 1]",1,1,1
2,2162049.jpg,"apple,banana,orange","[1, 1, 1]",1,1,1
3,6808904.jpg,"banana,orange,apple","[1, 1, 1]",1,1,1
4,8049862.jpg,banana,"[0, 0, 1]",0,0,1


<IPython.core.display.Javascript object>

In [12]:
gt_df["index"] = gt_df.index + 1
gt_df.head()

,image,class,class2,apple,orange,banana,index
0,5553944.jpg,banana,"[0, 0, 1]",0,0,1,1
1,9985835.jpg,"apple,orange,banana","[1, 1, 1]",1,1,1,2
2,2162049.jpg,"apple,banana,orange","[1, 1, 1]",1,1,1,3
3,6808904.jpg,"banana,orange,apple","[1, 1, 1]",1,1,1,4
4,8049862.jpg,banana,"[0, 0, 1]",0,0,1,5


<IPython.core.display.Javascript object>

In [13]:
sel_cols = ["index"] + labels + ["image"]
print(sel_cols)

['index', 'apple', 'orange', 'banana', 'image']


<IPython.core.display.Javascript object>

In [21]:
gt_df[sel_cols].head(ntrain).to_csv(
    "prepare_data/img_clf_multilabel_lst/train_annots/train.lst",
    sep="\t",
    index=False,
    header=False,
)
!head -n 5 prepare_data/img_clf_multilabel_lst/train_annots/train.lst

1	0	0	1	5553944.jpg
2	1	1	1	9985835.jpg
3	1	1	1	2162049.jpg
4	1	1	1	6808904.jpg
5	0	0	1	8049862.jpg


<IPython.core.display.Javascript object>

In [22]:
gt_df[sel_cols].tail(nvalid).to_csv(
    "prepare_data/img_clf_multilabel_lst/valid_annots/valid.lst",
    sep="\t",
    index=False,
    header=False,
)
!head -n 5 prepare_data/img_clf_multilabel_lst/valid_annots/valid.lst

71	1	1	0	2632491.jpg
72	1	0	0	8704016.jpg
73	1	1	0	3315941.jpg
74	1	1	0	7593329.jpg
75	1	1	0	7411721.jpg


<IPython.core.display.Javascript object>

In [16]:
train_df = pd.read_csv(
    "prepare_data/img_clf_multilabel_lst/train_annots/train.lst", sep="\t", header=None
)
images = list(train_df[train_df.columns[-1]].values)
for image in images:
    shutil.copy(
        "prepare_data/img_clf_multilabel_lst/all_images_gt/" + image,
        "prepare_data/img_clf_multilabel_lst/train_imgs/",
    )

<IPython.core.display.Javascript object>

In [17]:
valid_df = pd.read_csv(
    "prepare_data/img_clf_multilabel_lst/valid_annots/valid.lst", sep="\t", header=None
)
images = list(train_df[train_df.columns[-1]].values)
for image in images:
    shutil.copy(
        "prepare_data/img_clf_multilabel_lst/all_images_gt/" + image,
        "prepare_data/img_clf_multilabel_lst/valid_imgs/",
    )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

9314850.jpg,"[banana,apple]"
6799149.jpg,[apple]
9741355.jpg,"[banana,apple]"
3811265.jpg,"[orange,banana,apple]"
5260672.jpg,"[orange,banana]"
5915380.jpg,"[apple,banana]"
9421465.jpg,[apple]
3315941.jpg,"[apple,orange]"
9410453.jpg,"[banana,orange]"
8999964.jpg,[apple]


<IPython.core.display.Javascript object>